In [18]:
import requests
from bs4 import BeautifulSoup

In [19]:
# Define the URL of the website to scrape
url = "http://student.mit.edu/catalog/index.cgi"

# Send a GET request to the URL
response = requests.get(url)

# Create a BeautifulSoup object to parse the HTML content
soup = BeautifulSoup(response.content, "html.parser")

In [20]:
# Find all the course links
ul_element = soup.find("ul")

li_elements = ul_element.find_all("li")

In [28]:
links = ["http://student.mit.edu/catalog/m21Wa.html", "http://student.mit.edu/catalog/m21Wb.html"]

pagination = "bcdefghijklmnopqrstuvwxyz"
# Iterate over the li elements and extract the links
for li in li_elements:
    link = li.find("a")
    if link:
        links.append(f"http://student.mit.edu/catalog/{link.get('href')}")
    try:
        paginated_link = list(link.get("href"))
        for letter in pagination:
            paginated_link[-6] = letter
            res = requests.get(
                f"http://student.mit.edu/catalog/{''.join(paginated_link)}")

            if res.status_code == 200:
                # print(res.status_code)
                links.append(
                    f"http://student.mit.edu/catalog/{''.join(paginated_link)}")
            else:
                # print(letter)
                break
    except:
        pass

In [29]:
#  iterate over the links and save all courses in a list
# courses = set()
courses = {}
for link in links:
    response = requests.get(link)
    soup = BeautifulSoup(response.content, "html.parser")
    table = soup.find("table", width="100%", border="0")
#     print(link)
    a_tags = table.find_all("a", attrs={"name": True})
    
    for a_tag in a_tags:
        class_num = a_tag["name"]
        next_tag = a_tag.find_next()
        
        # Find the HASS/CI attributes (if present)
        attributes = next_tag.find_all('img')

        # Extract the attribute values from the 'alt' attribute of the image tags
        attribute_values = [attr.get('alt', '') for attr in attributes]

        # Store the class number and its attributes in the dictionary
        courses[class_num] = attribute_values
    
    
#     for a_tag in a_tags:
#         # print(a_tag["name"])
#         courses.add(a_tag["name"])

In [30]:
print(len(courses))

5669


In [33]:
courses

{'21W.011': ['______',
  'Undergrad',
  'Fall',
  'Spring',
  'HASS Humanities',
  'Communication Intensive Writing',
  'Add to schedule',
  '______'],
 '21W.012': ['______',
  'Undergrad',
  'Fall',
  'HASS Humanities',
  'Communication Intensive Writing',
  'Add to schedule',
  '______'],
 '21W.013': ['______',
  'Undergrad',
  'Fall',
  'HASS Humanities',
  'Communication Intensive Writing',
  'Add to schedule',
  '______'],
 '21W.014': ['______',
  'Undergrad',
  'Spring',
  'HASS Humanities',
  'Communication Intensive Writing',
  '______'],
 '21W.015': ['______',
  'Undergrad',
  'Fall',
  'Spring',
  'HASS Humanities',
  'Communication Intensive Writing',
  'Add to schedule',
  '______'],
 '21W.016': ['______',
  'Undergrad',
  'Fall',
  'HASS Humanities',
  'Communication Intensive Writing',
  'Add to schedule',
  '______'],
 '21W.021': ['______',
  'Undergrad',
  'Fall',
  'HASS Humanities',
  'Communication Intensive Writing',
  'Add to schedule',
  '______'],
 '21W.022': ['_

In [37]:
import json

# Mapping of attribute codes to their respective keys in the JSON
attribute_mapping = {
    'HASS Humanities': 'hh',
    'HASS Arts': 'ha',
    'HASS Social Sciences': 'hs',
    'HASS Elective': 'he',
    'Communication Intensive HASS': 'ci',
    'Communication Intensive Writing': 'cw'
}

# Convert the courses dictionary into the desired JSON format
json_data = {}
for course_code, course_info in courses.items():
    json_data[course_code] = {
        "no": course_code,
        "hh": "HASS Humanities" in course_info,
        "ha": "HASS Arts" in course_info,
        "hs": "HASS Social Sciences" in course_info,
        "he": "HASS Elective" in course_info,
        "ci": "Communication Intensive HASS" in course_info,
        "cw": "Communication Intensive Writing" in course_info
    }

# Save the JSON data to a file
with open("course_data.json", "w") as file:
    json.dump(json_data, file, indent=4)